In [2]:
import os

dependencies_dir = "./dependencies"
os.makedirs(dependencies_dir, exist_ok=True)

In [6]:
%%writefile {dependencies_dir}/conda.yaml
name: model-env
channels:
  - conda-forge
  - defaults
dependencies:
  - python=3.8
  - numpy>=1.21
  - pip
  - scikit-learn>=1.0
  - scipy>=1.7
  - pandas>=1.1,<1.2
  - pip:
    - tensorflow>=2.9,<2.12
    - mlflow
    - azureml-mlflow
    - azureml-defaults
    - inference-schema[numpy-support]
    - matplotlib
    - psutil
    - tqdm
    - ipykernel

Overwriting ./dependencies/conda.yaml


In [12]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import (
    Model, Environment, ManagedOnlineEndpoint, ManagedOnlineDeployment
)
from azure.ai.ml.constants import AssetTypes
import datetime

# authenticate
credential = DefaultAzureCredential()

# Set up MLClient
SUBSCRIPTION = "1a15ff42-0f80-4a67-94c1-7935c0ed9bd4"
RESOURCE_GROUP = "515LAB5"
WS_NAME = "WAND"
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

# Register the model
model = ml_client.models.create_or_update(
    Model(
        path="wand_model.h5",
        name="wand-gesture-model",
        type=AssetTypes.CUSTOM_MODEL,
        description="Trained gesture model for wand detection"
    )
)

# Create or reuse an environment
env = Environment(
    name="wand-inference-env",
    description="Inference environment for wand model",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
    conda_file="dependencies/conda.yaml",
)
env = ml_client.environments.create_or_update(env)

# Create unique endpoint name using timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
endpoint_name = f"wand-endpoint-{timestamp}"

# Define the endpoint
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Wand gesture classification endpoint",
    auth_mode="key"
)
ml_client.begin_create_or_update(endpoint).result()

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://wand-endpoint-20250607070104.westus2.inference.ml.azure.com/score', 'openapi_uri': 'https://wand-endpoint-20250607070104.westus2.inference.ml.azure.com/swagger.json', 'name': 'wand-endpoint-20250607070104', 'description': 'Wand gesture classification endpoint', 'tags': {}, 'properties': {'createdBy': 'Shiyi Chen', 'createdAt': '2025-06-07T07:01:07.949629+0000', 'lastModifiedAt': '2025-06-07T07:01:07.949629+0000', 'azureml.onlineendpointid': '/subscriptions/1a15ff42-0f80-4a67-94c1-7935c0ed9bd4/resourcegroups/515lab5/providers/microsoft.machinelearningservices/workspaces/wand/onlineendpoints/wand-endpoint-20250607070104', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/1a15ff42-0f80-4a67-94c1-7935c0ed9bd4/providers/Microsoft.MachineLearningServices/locations/westus2/mfeOperationsStatus/oeidp:36ef26e6-24d6-4de8-a84d-1ebf01c6fb06:55c1df62-e3ad-4cc7-ad9